# Identifying the base report

You are tasked with building the following visualization:

<center><img src="images/01.02.png"  style="width: 400px, height: 300px;"/></center>


A base report is an underlying report that sources a visualization. From a SQL point of view, you first want to build a base report before creating the visualization.

Which of the following statements about the base report for this visualization is false?

- The query needs a `WHERE` statement.

# Building the base report

Now, build the base report for this visualization:

<center><img src="images/01.02.png"  style="width: 400px, height: 300px;"/></center>

This should be built by querying the `summer_games` table.

```
-- Query the sport and distinct number of athletes
SELECT 
	sport, 
    COUNT(DISTINCT athlete_id) AS athletes
FROM summer_games
GROUP BY sport
-- Only include the 3 sports with the most athletes
ORDER BY athletes
LIMIT 3;

```

# Athletes vs events by sport

Now consider the following visualization:

<center><img src="images/01.04.png"  style="width: 400px, height: 300px;"/></center>


Using the summer_games table, run a query that creates the base report that sources this visualization.

```
-- Query sport, events, and athletes from summer_games
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events, 
    COUNT(DISTINCT athlete_id) AS athletes
FROM summer_games
GROUP BY sport;
```

# Planning queries with an E:R diagram

An E:R diagram visually shows all tables, fields, and relationships in a database. You are given the following E:R diagram:

<center><img src="images/01.06.png"  style="width: 400px, height: 300px;"/></center>

You are tasked with building a report that shows Age of Oldest Athlete by Region. What tables will need to be included in the query?

- `athletes`, `summer_games`, `countries`

# Age of oldest athlete by region

You are given the following E:R diagram:

<center><img src="images/01.06.png"  style="width: 400px, height: 300px;"/></center>

In the previous exercise, you identified which tables are needed to create a report that shows Age of Oldest Athlete by Region. Now, set up the query to create this report.


```
-- Select the age of the oldest athlete for each region
SELECT 
	region, 
    MAX(age) AS age_of_oldest_athlete
FROM athletes AS a
-- First JOIN statement
JOIN summer_games AS s
ON a.id = s.athlete_id
-- Second JOIN statement
JOIN countries AS c
ON c.id = s.country_id
GROUP BY region;
```

# Number of events in each sport

The full E:R diagram for the database is shown below:

<center><img src="images/01.08.png"  style="width: 400px, height: 300px;"/></center>


Since the company will be involved in both summer sports and winter sports, it is beneficial to look at all sports in one centralized report.

Your task is to create a query that shows the unique number of events held for each sport. Note that since no relationships exist between these two tables, you will need to use a UNION instead of a JOIN.

```
-- Select sport and events for summer sports
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events
FROM summer_games
GROUP BY sport
UNION
-- Select sport and events for winter sports
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events
FROM winter_games
GROUP BY sport
-- Show the most events at the top of the report
ORDER BY events DESC;
```

# Exploring summer_games

Exploring the data in a table can provide further insights into the database as a whole. In this exercise, you will try out a series of different techniques to explore the `summer_games` table.

```
-- Update query to explore the unique bronze field values
SELECT DISTINCT bronze
FROM summer_games;
```

```
-- Recreate the query by using GROUP BY 
SELECT bronze
FROM summer_games
GROUP BY bronze;
```

```
-- Add the rows column to your query
SELECT 
	bronze, 
	COUNT(*) AS rows
FROM summer_games
GROUP BY bronze;
```

# Validating our query

The same techniques we use to explore the data can be used to validate queries. By using the query as a subquery, you can run exploratory techniques to confirm the query results are as expected.

In this exercise, you will create a query that shows Bronze Medals by Country and then validate it using the subquery technique.

Feel free to reference the E:R Diagram as needed.

```
-- Pull total_bronze_medals from summer_games below
SELECT SUM(bronze) AS total_bronze_medals
FROM summer_games;
```

```
-- Setup a query that shows bronze_medal by country
SELECT 
	country, 
    SUM(bronze) AS bronze_medals
FROM summer_games AS s
JOIN countries AS c
ON c.id = s.country_id
GROUP BY country;
```

```
/* Pull total_bronze_medals below
SELECT SUM(bronze) AS total_bronze_medals
FROM summer_games; 
>> OUTPUT = 141 total_bronze_medals */

-- Select the total bronze_medals from your query
SELECT SUM(temp.bronze_medals)
FROM (
-- Previous query is shown below.  Alias this AS subquery
  SELECT 
      country, 
      SUM(bronze) AS bronze_medals
  FROM summer_games AS s
  JOIN countries AS c
  ON s.country_id = c.id
  GROUP BY country) AS temp
;
```

# Report 1: Most decorated summer athletes

Now that you have a good understanding of the data, let's get back to our case study and build out the first element for the dashboard, Most Decorated Summer Athletes:

<center><img src="images/01.12.png"  style="width: 400px, height: 300px;"/></center>


Your job is to create the base report for this element. Base report details:

- Column 1 should be athlete_name.
- Column 2 should be gold_medals.
- The report should only include athletes with at least 3 medals.
- The report should be ordered by gold medals won, with the most medals at the top.

```
-- Pull athlete_name and gold_medals for summer games
SELECT 
	name AS athlete_name, 
    SUM(gold) AS gold_medals
FROM summer_games AS s
JOIN athletes AS a
ON a.id = s.athlete_id
GROUP BY athlete_name
-- Filter for only athletes with 3 gold medals or more
HAVING SUM(gold) >= 3
-- Sort to show the most gold medals at the top
ORDER BY gold_medals DESC;
```

# Identifying duplication

Duplication can happen for a number of reasons, often in unexpected ways. Because of this, it's important to get in the habit of validating your queries to ensure no duplication exists. To validate a query, take the following steps:

- Check the total value of a metric from the original table.
- Compare that with the total value of the same metric in your final report.

If the number from step 2 is larger than step 1, then duplication is likely the culprit. In this exercise, you will go through these steps to identify if duplication exists.

```
-- Comment out the query after noting the gold medal count
/*SELECT SUM(gold) AS gold_medals
FROM winter_games;*/
-- TOTAL GOLD MEDALS: 47 

-- Calculate the total gold_medals in your query
SELECT SUM(gold_medals)
FROM
	(SELECT 
        w.country_id, 
     	SUM(gold) AS gold_medals, 
        AVG(gdp) AS avg_gdp
    FROM winter_games AS w
    JOIN country_stats AS c
    ON c.country_id = w.country_id
    -- Alias your query as subquery
    GROUP BY w.country_id) AS subquery;
```

# Fixing duplication through a JOIN

In the previous exercise, you set up a query that contained duplication. This exercise will remove the duplication. One approach to removing duplication is to change the `JOIN` logic by adding another field to the `ON` statement.

The final query from last exercise is shown in the console. Your job is to fix the duplication by updating the `ON` statement. Note that the total `gold_medals` value should be 47.

Feel free to reference the E:R Diagram.

```
SELECT SUM(gold_medals) AS gold_medals
FROM
	(SELECT 
     	w.country_id, 
     	SUM(gold) AS gold_medals, 
     	AVG(gdp) AS avg_gdp
    FROM winter_games AS w
    JOIN country_stats AS c
    -- Update the subquery to join on a second field
    ON c.country_id = w.country_id AND CAST(c.year AS DATE) = w.year
    GROUP BY w.country_id) AS subquery;
```

# Report 3: Countries with high medal rates

Great work so far! It is time to use the concepts you learned in this chapter to build the next base report for your dashboard.

Details for report 3: medals vs population rate.

- Column 1 should be `country_code`, which is an altered version of the `country` field.
- Column 2 should be `pop_in_millions`, representing the population of the country (in millions).
- Column 3 should be `medals`, representing the total number of medals.
- Column 4 should be `medals_per_million`, which equals `medals` / `pop_in_millions`

```
SELECT 
	c.country,  
    -- Add the three medal fields using one sum function
	SUM(COALESCE(bronze,0)+COALESCE(silver,0)+COALESCE(gold,0)) AS medals
FROM summer_games AS s
JOIN countries AS c
ON c.id = s.country_id
GROUP BY c.country
ORDER BY medals DESC;
```

```
SELECT 
	c.country,
    -- Pull in pop_in_millions and medals_per_million 
	pop_in_millions,
    -- Add the three medal fields using one sum function
	SUM(COALESCE(bronze,0) + COALESCE(silver,0) + COALESCE(gold,0)) AS medals,
	SUM(COALESCE(bronze,0) + COALESCE(silver,0) + COALESCE(gold,0)) / CAST(cs.pop_in_millions AS float) AS medals_per_million
FROM summer_games AS s
JOIN countries AS c 
ON s.country_id = c.id
-- Update the newest join statement to remove duplication
JOIN country_stats AS cs 
ON s.country_id = cs.country_id AND c.id=cs.country_id AND s.year = CAST(cs.year AS DATE)
GROUP BY c.country, pop_in_millions
ORDER BY medals DESC;
```

```
SELECT 
	-- Clean the country field to only show country_code
    LEFT(REPLACE(UPPER(TRIM(c.country)),'.',''),3) AS country_code,
    -- Pull in pop_in_millions and medals_per_million 
	pop_in_millions,
    -- Add the three medal fields using one sum function
	SUM(COALESCE(bronze,0) + COALESCE(silver,0) + COALESCE(gold,0)) AS medals,
	SUM(COALESCE(bronze,0) + COALESCE(silver,0) + COALESCE(gold,0)) / CAST(cs.pop_in_millions AS float) AS medals_per_million
FROM summer_games AS s
JOIN countries AS c 
ON s.country_id = c.id
-- Update the newest join statement to remove duplication
JOIN country_stats AS cs 
ON s.country_id = cs.country_id AND s.year = CAST(cs.year AS date)
-- Filter out null populations
WHERE pop_in_millions IS NOT NULL
GROUP BY c.country, pop_in_millions
-- Keep only the top 25 medals_per_million rows
ORDER BY medals_per_million DESC
LIMIT 25;
```